In [286]:
import os
import json
import numpy as np
import tensorflow as tf

In [287]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [288]:
# Name of directory for grids
directory_name = './data/big_grids/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_distance.append(i['distance'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()

# direct_count = [0 for i in range(4)]
# sample = []

# # iterate through all training grids
# for file_name in os.listdir(directory_name):
#     f = open(directory_name + file_name)
#     data = json.load(f)

#     # Iterate through all the data in a given grid and append their input and output values
#     for i in data:
#         if direct_count[i['direction']] < 10000:
#             sample.append(i)
#             direct_count[i['direction']] += 1
    
#     # Close file socket
#     f.close()

# np.random.shuffle(sample)
# for i in sample:
#     x_train_grid.append(i['gridworld'])
#     x_train_position.append(i['position'])
#     x_train_distance.append(i['distance'])
#     x_train_locals.append(i['local'])
#     y_train.append(i['direction'])
    

In [289]:
#print(np.reshape(x_train_grid[25], (30,30)))
#print(x_train_position[25])
#print(y_train[25])

In [290]:
print(direct_count)

[7324, 17775, 42744, 63171]


In [291]:
# Name of directory for grids
directory_name = './data/big_grids_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_distance.append(i['distance'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [292]:
#print(x_train_position)
#print(y_train)

In [293]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 50, 50) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 2, 1) )
train_in_distance = np.reshape( x_train_distance, (-1,1,1) ) / 60
train_in_locals = np.reshape( x_train_locals, (-1, 3, 3) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 50, 50) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 2, 1) )
test_in_distance = np.reshape( x_test_distance, (-1, 1, 1) ) / 60
test_in_locals = np.reshape( x_test_locals, (-1, 3, 3) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [294]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (50,50) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [295]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [296]:
# flatten the distance inputs
distance_input = tf.keras.layers.Input( shape = (1,1) )
flatten_distance = tf.keras.layers.Flatten()( distance_input )

In [297]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (3,3) )
flatten_local = tf.keras.layers.Flatten()( local_input )

In [298]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_local, flatten_position])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 2511), dtype=tf.float32, name=None), name='concatenate_14/concat:0', description="created by layer 'concatenate_14'")


In [299]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 80, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 30, activation = tf.nn.relu )( dense_2 )
dense_4 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_3 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_4 )
probabilities = tf.keras.layers.Softmax()( logits )

In [300]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, local_input, position_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [301]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [302]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

0	0	1219	0
0	0	2958	0
0	0	7144	0
0	0	10518	0


In [303]:
# Train the model
history = model.fit( [train_in_grid, train_in_locals, train_in_position], train_out, validation_data=([test_in_grid, test_in_locals, test_in_position], test_out), epochs = 10 )

Epoch 1/10
4095/4095 [==============================] - 20s 5ms/step - loss: 0.9254 - accuracy: 0.6148 - val_loss: 0.7006 - val_accuracy: 0.7482
Epoch 2/10
4095/4095 [==============================] - 17s 4ms/step - loss: 0.6460 - accuracy: 0.7472 - val_loss: 2.6352 - val_accuracy: 0.2126
Epoch 3/10
4095/4095 [==============================] - 16s 4ms/step - loss: 0.5795 - accuracy: 0.7746 - val_loss: 0.7125 - val_accuracy: 0.6872
Epoch 4/10
4095/4095 [==============================] - 18s 4ms/step - loss: 0.4779 - accuracy: 0.8125 - val_loss: 0.3576 - val_accuracy: 0.8571
Epoch 5/10
4095/4095 [==============================] - 18s 4ms/step - loss: 0.3961 - accuracy: 0.8509 - val_loss: 0.2805 - val_accuracy: 0.9063
Epoch 6/10
4095/4095 [==============================] - 18s 4ms/step - loss: 0.4097 - accuracy: 0.8480 - val_loss: 0.4351 - val_accuracy: 0.8516
Epoch 7/10
4095/4095 [==============================] - 18s 4ms/step - loss: 0.3780 - accuracy: 0.8636 - val_loss: 0.2896 - val_ac

In [304]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

224	789	192	14
42	2678	226	12
17	27	7061	39
3	178	82	10255


In [306]:
model.save('./models/agent1_NN')

INFO:tensorflow:Assets written to: ./models/agent1_NN/assets
